### Feature Extraction
* **Products used:** 
[dem_cop_30](https://explorer.digitalearth.africa/products/s2_l2a), [s2_l2a](https://explorer.digitalearth.africa/products/dem_cop_90), [dem_srtm](https://explorer.digitalearth.africa/products/dem_srtm), [dem_srtm_deriv](https://explorer.digitalearth.africa/products/dem_srtm_deriv)

## Background:

Training data extraction plays a crucial role in training machine learning models. The process involves extracting relevant feature layers from a geospatial dataset based on predefined geometries or regions of interest. This enables the creation of accurate and reliable classification models for various applications such as land cover mapping, crop monitoring, and environmental analysis.

To facilitate this task, the open-data-cube provides a powerful function called "collect_training_data." This function is part of the deafrica_tools.classification script and is specifically designed to extract training data from the open-data-cube using geometries defined within a GeoJSON file. The GeoJSON file contains the spatial boundaries or polygons that delineate the regions of interest for which training data needs to be extracted.

## Description:

This notebook focuses on the extraction of training data (feature layers) from the open-data-cube using geometries defined within a GeoJSON file. It follows a step-by-step approach to guide users in utilizing the "collect_training_data" function effectively. The goal is to enable users to extract the appropriate training data for their specific use case.

The main steps in this notebook are as follows:

1. **Previewing the Training Data:** The notebook starts by plotting the polygons from the training data on a basemap. This visualization provides users with a visual representation of the regions of interest for which training data will be extracted.

2. **Defining the Feature Layer Function:** Next, a feature layer function is defined. This function specifies the set of feature layers to be extracted from the open-data-cube. These layers are carefully selected based on their relevance to the classification task at hand.

3. **Extracting Training Data:** The "collect_training_data" function is then employed to extract the training data from the datacube. It utilizes the predefined geometries from the GeoJSON file and retrieves the corresponding feature layers. This step ensures that the extracted data aligns precisely with the defined regions of interest.

4. **Exporting Training Data:** Finally, the extracted training data is exported and saved to disk. This facilitates its subsequent use in other scripts or machine learning workflows for training classification models.

By following the steps outlined in this notebook, users can leverage the "collect_training_data" function to efficiently extract training data from the open-data-cube. 

## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load packages

In [1]:
%matplotlib inline
import io
import os 
import math
import datacube
import warnings
import rioxarray
import rasterio
import numpy as np
import pandas as pd
import xarray as xr
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt

from datacube.testutils.io import rio_slurp_xarray


from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import map_shapefile
from deafrica_tools.bandindices import calculate_indices
from classification import collect_training_data


## Analysis parameters
 * path: The path to the input vector file from which we will extract training data. A default geojson is provided.
 * field: This is the name of column in your shapefile attribute table that contains the class labels. The class labels must be integers

In [2]:
# Specify a prefix to identify the area of interest in the saved outputs
# By assigning the desired prefix, you can easily identify the outputs associated with the specific area of interest.
SKYCOMISH_HUC8_ID = "17110009"
prefix = SKYCOMISH_HUC8_ID
field = "class_id"
path = f"data/{prefix}_training_samples.geojson"

# Load input data shapefile
training_points = gpd.read_file(path)
training_points.head()

,class_id,class_name,geometry
0,0,Non-wetland,POINT (-121.46688 47.82286)
1,0,Non-wetland,POINT (-121.38165 47.76169)
2,2,Freshwater Forested/Shrub Wetland,POINT (-121.29296 47.64969)
3,0,Non-wetland,POINT (-121.81442 47.8725)
4,0,Non-wetland,POINT (-121.75595 47.82459)


In [3]:
# Set a flag to convert to polygons:
use_polygons = False

if use_polygons:
    # Convert from lat,lon to EPSG:6933 (projection in metres)
    training_points = training_points.to_crs("EPSG:6933")

    # Buffer geometry to get a square - only if trying to sample multiple pixels
    buffer_radius_m = 10
    training_points.geometry = training_points.geometry.buffer(
        buffer_radius_m, cap_style=3
    )

#### Plot on interactive map 

In [4]:
points = training_points
training_points.explore(
    tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
    attr="Imagery @2022 Landsat/Copernicus, Map data @2022 Google",
    popup=True,
    cmap="viridis",
    style_kwds=dict(radius=5, color="red", fillOpacity=0.8, fillColor="red", weight=3),
)

In [9]:
dem_file = "../../../data/processed/17110009_Skykomish_HE_DEM_3m.tif"
CRS = rioxarray.open_rasterio(dem_file, chunks=True).squeeze(drop=True).odc.crs

In [10]:
from pathlib import Path
import rioxarray as rx


def load_training_data():
    return xr.merge(
        [
            rx.open_rasterio(f, chunks=True).squeeze(drop=True).to_dataset(name=f.stem)
            for f in Path("data/terrain_attributes").glob("*.tif")
        ]
    )


training_xr = load_training_data().rio.write_crs(CRS)

In [11]:
import pandas as pd
import xarray as xr


def sample_xr(xr_ds: xr.Dataset, points: gpd.GeoDataFrame):
    points_proj = points.to_crs(xr_ds.odc.crs)
    pts_da = points_proj.assign(
        x=points_proj.geometry.x, y=points_proj.geometry.y
    ).to_xarray()

    # a dataframe or series (for a single point)
    pt_values_i = (
        xr_ds.sel(pts_da[["x", "y"]], method="nearest").squeeze().compute().to_pandas()
    )

    if isinstance(pt_values_i, pd.Series):
        pt_values_i = pt_values_i.to_frame().transpose()
        pt_values_i.index = points.index

    return pd.concat([points_proj, pt_values_i], axis=1)


pd_training_features = sample_xr(training_xr, training_points)
pd_training_features

,class_id,class_name,geometry,x,y,mrvbf,TPI_453.0m,DTW,Profile_curvature_453.0m,mrrtf,...,slope,TWI,Planform_curvature_93.0m,Curvature_453.0m,Slope_273.0m,Curvature_93.0m,TPI_93.0m,Planform_curvature_453.0m,Profile_curvature_93.0m,spatial_ref
0,0,Non-wetland,POINT (-1898576.684 3014544.396),-1898577.5,3014543.5,0,26.666382,8.730106,0.316817,0,...,45.385658,-0.316005,3.636138,-0.345143,70.346046,-0.051668,-4.925537,-0.661960,-3.687806,0
1,0,Non-wetland,POINT (-1894156.246 3006332.771),-1894155.5,3006332.5,0,-3.847534,0.652631,0.750960,0,...,65.263123,1.402493,1.919346,-0.477897,63.156918,0.024647,-0.715271,-1.228857,-1.894699,0
2,2,Freshwater Forested/Shrub Wetland,POINT (-1890946.915 2992639.728),-1890945.5,2992640.5,4,-10.100922,1.174084,0.065914,0,...,3.105867,2.964397,0.022249,0.206286,4.006325,0.015298,-0.102875,0.140372,-0.006950,0
3,0,Non-wetland,POINT (-1922380.443 3026809.708),-1922379.5,3026810.5,0,-0.033524,1.404718,0.023109,3,...,1.341787,2.990659,-0.009691,0.026888,2.525811,0.002469,0.035912,0.003779,0.012160,0
4,0,Non-wetland,POINT (-1919548.662 3020535.488),-1919547.5,3020534.5,1,-7.737000,0.000000,0.199140,0,...,8.730982,11.741082,0.101506,0.410431,15.863617,-0.033583,-0.912872,0.211291,-0.135089,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0,Non-wetland,POINT (-1896366.705 3002368.93),-1896366.5,3002369.5,3,-2.653381,1.004033,0.055589,0,...,2.276607,2.547368,-0.020424,0.069693,1.213640,-0.028012,0.178131,0.014104,-0.007589,0
3996,2,Freshwater Forested/Shrub Wetland,POINT (-1903003.86 3001223.519),-1903002.5,3001223.5,2,-13.665955,0.641491,-0.022879,0,...,3.911239,2.060971,-0.142191,0.018532,9.654344,-0.032528,-0.526306,0.041411,0.109663,0
3997,0,Non-wetland,POINT (-1899914.976 3020630.107),-1899915.5,3020630.5,0,5.873779,29.410852,0.344652,0,...,101.416725,1.717248,4.979585,-0.478709,71.894241,0.381641,-0.362488,-0.823361,-4.597943,0
3998,0,Non-wetland,POINT (-1901618.626 3020589.191),-1901619.5,3020588.5,0,12.527466,12.806068,1.342011,0,...,116.418800,3.103205,-1.747261,0.441355,68.020256,-0.136493,0.674927,-0.900656,1.610768,0


### Export training features

In [12]:
# set the name and location of the output file
# output_file = "results/training_features.txt"
output_file = f"data/{prefix}_training_features.csv"
# Export files to disk
pd_training_features.to_csv(output_file, header=True, index=None, sep=" ")

In [13]:
# create geopandas dataframe
gpd_training_features = gpd.GeoDataFrame(
    pd_training_features,
    geometry="geometry",
)

#####  Add a column for binary (wetland/non-wetland) classification
This block ensures that both binary and multi-class classification labels are properly set up from the original `class_id` field:

- If `class_id` contains only two values (0 and 1), it is assumed to be binary, and is renamed to `class_id_binary`.
- If `class_id` includes additional wetland types, a new binary column `class_id_binary` is created:
  - `1` for any wetland type (i.e., values not equal to 0)
  - `0` for non-wetland (i.e., value equal to 0)
  - The original column is then renamed to `class_id_type` for use in multi-class classification.

In [14]:
# Check if unique values in 'class_id' are only 0 and 1
unique_values = gpd_training_features["class_id"].unique()
if len(unique_values) == 2 and set(unique_values) == {0, 1}:
    # Replace 'class_id' with 'class_id_binary'
    gpd_training_features.rename(columns={"class_id": "class_id_binary"}, inplace=True)
else:
    # Create 'class_id_binary' column based on condition
    gpd_training_features["class_id_binary"] = gpd_training_features["class_id"].apply(
        lambda x: 1 if x != 0 else 0
    )
    gpd_training_features.rename(columns={"class_id": "class_id_type"}, inplace=True)

# Insert the new column at the second position
gpd_training_features.insert(
    0, "class_id_binary", gpd_training_features.pop("class_id_binary")
)
print(gpd_training_features.columns)

Index(['class_id_binary', 'class_id_type', 'class_name', 'geometry', 'x', 'y',
       'mrvbf', 'TPI_453.0m', 'DTW', 'Profile_curvature_453.0m', 'mrrtf',
       'TPI_273.0m', 'Curvature_273.0m', 'Elevation',
       'Profile_curvature_273.0m', 'Slope_93.0m', 'Planform_curvature_273.0m',
       'Slope_453.0m', 'slope', 'TWI', 'Planform_curvature_93.0m',
       'Curvature_453.0m', 'Slope_273.0m', 'Curvature_93.0m', 'TPI_93.0m',
       'Planform_curvature_453.0m', 'Profile_curvature_93.0m', 'spatial_ref'],
      dtype='object')


In [15]:
# Replace non-zero values in the 'class_id' column with 1
gpd_training_features["class_id_binary"] = gpd_training_features["class_id_type"].apply(
    lambda x: 1 if x != 0 else 0
)
# Insert the new column at the second position
gpd_training_features.insert(
    1, "class_id_binary", gpd_training_features.pop("class_id_binary")
)
gpd_training_features

,class_id_type,class_id_binary,class_name,geometry,x,y,mrvbf,TPI_453.0m,DTW,Profile_curvature_453.0m,...,slope,TWI,Planform_curvature_93.0m,Curvature_453.0m,Slope_273.0m,Curvature_93.0m,TPI_93.0m,Planform_curvature_453.0m,Profile_curvature_93.0m,spatial_ref
0,0,0,Non-wetland,POINT (-1898576.684 3014544.396),-1898577.5,3014543.5,0,26.666382,8.730106,0.316817,...,45.385658,-0.316005,3.636138,-0.345143,70.346046,-0.051668,-4.925537,-0.661960,-3.687806,0
1,0,0,Non-wetland,POINT (-1894156.246 3006332.771),-1894155.5,3006332.5,0,-3.847534,0.652631,0.750960,...,65.263123,1.402493,1.919346,-0.477897,63.156918,0.024647,-0.715271,-1.228857,-1.894699,0
2,2,1,Freshwater Forested/Shrub Wetland,POINT (-1890946.915 2992639.728),-1890945.5,2992640.5,4,-10.100922,1.174084,0.065914,...,3.105867,2.964397,0.022249,0.206286,4.006325,0.015298,-0.102875,0.140372,-0.006950,0
3,0,0,Non-wetland,POINT (-1922380.443 3026809.708),-1922379.5,3026810.5,0,-0.033524,1.404718,0.023109,...,1.341787,2.990659,-0.009691,0.026888,2.525811,0.002469,0.035912,0.003779,0.012160,0
4,0,0,Non-wetland,POINT (-1919548.662 3020535.488),-1919547.5,3020534.5,1,-7.737000,0.000000,0.199140,...,8.730982,11.741082,0.101506,0.410431,15.863617,-0.033583,-0.912872,0.211291,-0.135089,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0,0,Non-wetland,POINT (-1896366.705 3002368.93),-1896366.5,3002369.5,3,-2.653381,1.004033,0.055589,...,2.276607,2.547368,-0.020424,0.069693,1.213640,-0.028012,0.178131,0.014104,-0.007589,0
3996,2,1,Freshwater Forested/Shrub Wetland,POINT (-1903003.86 3001223.519),-1903002.5,3001223.5,2,-13.665955,0.641491,-0.022879,...,3.911239,2.060971,-0.142191,0.018532,9.654344,-0.032528,-0.526306,0.041411,0.109663,0
3997,0,0,Non-wetland,POINT (-1899914.976 3020630.107),-1899915.5,3020630.5,0,5.873779,29.410852,0.344652,...,101.416725,1.717248,4.979585,-0.478709,71.894241,0.381641,-0.362488,-0.823361,-4.597943,0
3998,0,0,Non-wetland,POINT (-1901618.626 3020589.191),-1901619.5,3020588.5,0,12.527466,12.806068,1.342011,...,116.418800,3.103205,-1.747261,0.441355,68.020256,-0.136493,0.674927,-0.900656,1.610768,0


In [16]:
# save as geojson file
results_dir = Path("results")
os.makedirs(results_dir, exist_ok=True)
geojson_file = f"results/{prefix}_training_features.geojson"
gpd_training_features.to_file(geojson_file, driver="GeoJSON")

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:** 

In [ ]:
from datetime import datetime

datetime.today().strftime("%Y-%m-%d")